In [1]:
import os
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
from tqdm import tqdm

# LangChain 관련
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.faiss import FAISS
from langchain.schema import Document
import fitz  # PyMuPDF
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage

# EnsembleRetriever, KiwiBM25Retriever 추가
from langchain.retrievers import EnsembleRetriever
from langchain_teddynote.retrievers import KiwiBM25Retriever

# ollama_model_load.py에서 정의한 Ollama 모델 llama 불러오기
from ollama_model_load import llama

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# 1) 평가 지표 계산 함수
def calculate_bleu(reference: str, generated: str):
    reference_tokens = reference.split()
    generated_tokens = generated.split()
    smoothing_function = SmoothingFunction().method1
    return sentence_bleu([reference_tokens], generated_tokens, smoothing_function=smoothing_function)

def calculate_meteor(reference: str, generated: str):
    reference_tokens = reference.split()
    generated_tokens = generated.split()
    return meteor_score([reference_tokens], generated_tokens)

def calculate_rouge(reference: str, generated: str):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, generated)
    return {
        "rouge_1": scores["rouge1"].fmeasure,
        "rouge_2": scores["rouge2"].fmeasure,
        "rouge_L": scores["rougeL"].fmeasure
    }

# 2) RAG 검색 수행 함수 (EnsembleRetriever 적용)
def perform_rag(
    question: str,
    pdf_file: str = "QADataset_new.pdf",
    chunk_size: int = 100,
    chunk_overlap: int = 50,
    device: str = "cuda"
) -> str:
    doc = fitz.open(pdf_file)
    full_text = ""
    for page in doc:
        full_text += page.get_text()

    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = [Document(page_content=t) for t in splitter.split_text(full_text)]

    model_kwargs = {"device": device}
    encode_kwargs = {"normalize_embeddings": True}
    embeddings = HuggingFaceEmbeddings(
        model_name="intfloat/multilingual-e5-large",
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs,
    )

    # FAISS Retriever 생성
    db_faiss = FAISS.from_documents(chunks, embedding=embeddings)
    faiss_retriever = db_faiss.as_retriever(search_kwargs={"k": 2})

    # KiwiBM25Retriever 생성
    kiwi_bm25_retriever = KiwiBM25Retriever.from_documents(chunks)

    # EnsembleRetriever 생성
    retriever = EnsembleRetriever(
        retrievers=[kiwi_bm25_retriever, faiss_retriever],
        weights=[1, 0],
        search_type="mmr",
    )

    # 검색 결과 가져오기
    context_docs = retriever.get_relevant_documents(question)
    context_text = "\n\n".join(doc.page_content for doc in context_docs)
    return context_text

# 3) LLM 질의 함수 (RAG Prompt 포함)
def query_llm(context: str, question: str) -> str:
    RAG_PROMPT_TEMPLATE = """
    아래 정보(context)를 참고하여 사용자 질문에 답해주세요:
    {context}

    질문:
    {question}

    답변 시, 질문의 핵심만 파악하여 간결하게 1~2문장으로 답변하고, 
    불필요한 설명은 피합니다. (동서울대학교 관련 정보라면 그 내용만 요약)

    답변:
    """
    prompt = ChatPromptTemplate.from_template(RAG_PROMPT_TEMPLATE)
    formatted_prompt = prompt.format(context=context, question=question)
    message = HumanMessage(content=formatted_prompt)

    response = llama([message])  
    return response.content.strip()

# 4) 전체 평가 함수
def evaluate_model_responses(
    csv_file: str = "QADataset_old.csv",
    pdf_file: str = "QADataset_new.pdf",
    output_file: str = "Evaluation_RAG_results.csv",
    batch_size: int = 5,
    chunk_size: int = 100,
    chunk_overlap: int = 50,
    device: str = "cuda"
):
    processed_count = 0
    if os.path.exists(output_file):
        existing_df = pd.read_csv(output_file, encoding='utf-8-sig')
        processed_count = len(existing_df)

    try:
        df = pd.read_csv(csv_file, encoding='utf-8-sig')
    except UnicodeDecodeError:
        df = pd.read_csv(csv_file, encoding='euc-kr')
    except Exception as e:
        print(f"CSV 파일을 열 때 오류 발생: {e}")
        return pd.DataFrame()

    total_rows = len(df)
    if processed_count >= total_rows:
        print("이미 모든 행이 처리되었습니다.")
        return pd.read_csv(output_file, encoding='utf-8-sig')

    evaluation_results = []
    for idx in tqdm(range(processed_count, total_rows), desc="Evaluating with RAG"):
        question = df.iloc[idx, 0]
        reference = df.iloc[idx, 1]

        context = perform_rag(
            question=question,
            pdf_file=pdf_file,
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            device=device
        )

        generated_response = query_llm(context, question)

        bleu_score = calculate_bleu(reference, generated_response)
        meteor_score_value = calculate_meteor(reference, generated_response)
        rouge_scores = calculate_rouge(reference, generated_response)

        evaluation_results.append({
            "question": question,
            "reference": reference,
            "generated": generated_response,
            "bleu": bleu_score,
            "meteor": meteor_score_value,
            "rouge_1": rouge_scores["rouge_1"],
            "rouge_2": rouge_scores["rouge_2"],
            "rouge_L": rouge_scores["rouge_L"]
        })

        if (len(evaluation_results) % batch_size == 0) or (idx == total_rows - 1):
            partial_df = pd.DataFrame(evaluation_results)

            if os.path.exists(output_file) and processed_count > 0:
                partial_df.to_csv(
                    output_file,
                    mode='a',
                    index=False,
                    header=False,
                    encoding='utf-8-sig'
                )
            else:
                partial_df.to_csv(
                    output_file,
                    mode='w',
                    index=False,
                    header=True,
                    encoding='utf-8-sig'
                )

            evaluation_results = []
            processed_count = idx + 1

    final_df = pd.read_csv(output_file, encoding='utf-8-sig')
    print(f"평가 완료! 결과는 '{output_file}'에 저장되었습니다.")
    return final_df

# 5) 메인 실행 예시
if __name__ == "__main__":
    # RAG 평가 실행
    final_df = evaluate_model_responses(
        csv_file="QADataset_new.csv",     
        pdf_file="QADataset_new.pdf",     
        output_file="llama_NonePEFT_RAG_new.csv", 
        batch_size=5,
        chunk_size=100,
        chunk_overlap=50,
        device="cuda"
    )
    
    print(final_df.head())


Evaluating with RAG:   0%|          | 0/421 [00:00<?, ?it/s]/tmp/ipykernel_3902077/763786023.py:65: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/home/du3/Desktop/duchatbot/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_3902077/763786023.py:86: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
 

평가 완료! 결과는 'llama_NonePEFT_RAG_new.csv'에 저장되었습니다.
                             question  \
0                  산업디자인학과는 어떤 학과인가요?   
1           산업디자인학과의 주요 학과 연혁은 무엇인가요?   
2  산업디자인학과의 교수님들은 누구이며, 연락처는 어떻게 되나요?   
3           산업디자인학과의 교육목표와 방침은 무엇인가요?   
4           산업디자인학과 졸업 후 진로는 어떻게 되나요?   

                                           reference  \
0  산업디자인학과는 창의적이고 능동적인 산업디자이너를 양성하며, 제품디자인, UI/UX...   
1  1981년에 학과가 신설되었으며, 2021년 산업디자인학과로 명칭이 변경되었습니다....   
2  1. 유승용 교수: 본관 5층 M503호, 031-720-2116, acapsule...   
3  융복합 기술 시대에 대응한 현장 중심의 직무능력과 창의적 사고력을 길러내어, 국내외...   
4  제품디자인, 환경디자인, 3D프린팅디자인, 멀티미디어디자인, UI/UX디자인 등 다...   

                                           generated      bleu    meteor  \
0                                    صن業디자인학과가 있습니다.  0.000000  0.000000   
1  산업디자인학과는 교육부 선정 사회맞춤형 사업의 일환으로 5년간 운영하여 현장중심의 ...  0.000000  0.000000   
2  산업디자인학과의 교수님들은 유승용교수님, 이경아교수님, 다까스요꼬교수님, 김두한교수...  0.003051  0.055556   
3  산업디자인학과의 교육목표는 4차 산업 혁신과 융합지식을 발전시키기 위해 항공및설계분...  0.000000